In [1]:
#Google Driveのマウント
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
path = '/content/gdrive/MyDrive/プログラミング用フォルダ/ヤフショ用python/在庫更新分割'

In [5]:
#モジュールのインポート
import os
import shutil
import glob
import pandas as pd
import numpy as np

#設定
delivery_line = 921 #送料ライン
low_price = 1498 #最低価格
delivery = 100 #国内配送料
profit_rate_nomal = 1.15 #利益率
profit_rate_sale = 1.2 #利益率
tax = 1.04 #関税・消費税
commission = 1-0.08 #手数料

# #ファイルの読み込み
# file_path = glob.glob('./在庫csv/*txt.csv')[-1]
file_path = glob.glob(os.path.join(path,'在庫csv/*txt.csv'))[-1]
df = pd.read_csv(file_path, encoding="shift_jis")

#価格の計算
fba = df['最安値[FBA]']
df.loc[(fba != 0) & (fba > delivery_line) , 'price'] = (
    (((fba*profit_rate_nomal*tax+delivery)/commission)//100*100))

df.loc[(df['price'] > low_price) & (df['price'] < 9998), 'price'] = (
    df['price']+(1-df['price']//100%2)*100+98)

df.loc[df['price'] > 9998, 'price'] = (
    df['price']//1000*1000+(1-df['price']//1000%2)*1000+980)

df.loc[df['price'] < 1598, 'price'] = 1598
df.loc[(fba <= 921) & (fba != 0), 'price'] = low_price
df.loc[fba == 0, 'price'] = 0

df['sale-price'] = ''
df['sale-period-start'] = ''
df['sale-period-end'] = ''


#小文字kk00化
ser_code = df['ASIN'].str.lower() + ('kk00')
ser_price = df['price'].astype('int')
ser_sale_price = df['sale-price']
ser_sale_period_start = df['sale-period-start']
ser_sale_period_end = df['sale-period-end']



#series作成
df_00 = pd.DataFrame(data={
    'code':ser_code,
    'price':ser_price,
    'sale-price':ser_sale_price,
    'sale-period-start':ser_sale_period_start,
    'sale-period-end':ser_sale_period_end
})

df_00.loc[df_00['price'] == 0, 'display'] = 0
df_00.loc[df_00['price'] != 0, 'display'] = 1
ser_display = df_00['display'].astype('int')
df_00.loc[df_00['price'] == 0, 'lead-time-instock'] = 3
df_00.loc[df_00['price'] != 0, 'lead-time-instock'] = 1
ser_lead_time_instock = df_00['lead-time-instock'].astype('int')
#送料あり[3]、送料なし[1]
df_00.loc[df_00['price'] <= 1499, 'delivery'] = 3
df_00.loc[df_00['price'] > 1499, 'delivery'] = 1
ser_delivery = df_00['delivery'].astype('int')


#DataFrame作成
df_01 = pd.DataFrame(data={
    'code':ser_code,
    'price':ser_price,
    'sale-price':ser_sale_price,
    'sale-period-start':ser_sale_period_start,
    'sale-period-end':ser_sale_period_end,
    'display':ser_display,
    'lead-time-instock':ser_lead_time_instock,
    'delivery':ser_delivery
})


#0の空白化
df_01.loc[df_00['price'] == 0, 'price'] = ''


# #ファイス分割
# for i in range(len(df_01)//100000+1):
#     df_01[(i+1)*100000-100000:(i+1)*100000].to_csv(
#         path_or_buf=f'./分割ファイル/{i+71}_価格変更.csv', 
#         index=False, encoding="shift_jis")

#ファイス分割
for i in range(len(df_01)//100000+1):
    df_01[(i+1)*100000-100000:(i+1)*100000].to_csv(
        path_or_buf=f'{path}/分割ファイル/{i+71}_価格変更.csv', 
        index=False, encoding="shift_jis")
    

#在庫切れの割合表示
df_bool = (df['最安値[FBA]'] == 0)
zaikogire_ritu = df_bool.sum()/len(df_bool)*100
print(f'{zaikogire_ritu:.1f}')
print(file_path)


df_quan = pd.DataFrame(data={
    'code':ser_code,
    'quantity':3
    })

#ファイス分割
for i in range(len(df_quan)//100000+1):
    df_quan[(i+1)*100000-100000:(i+1)*100000].to_csv(
        path_or_buf=f'{path}/分割ファイル/{i+91}_在庫設定.csv', 
        index=False, encoding="shift_jis")

40.9
/content/gdrive/MyDrive/プログラミング用フォルダ/ヤフショ用python/在庫更新分割/在庫csv/PRIMA_JAPAN_221113_153857_ASIN List2.txt.csv


In [ ]:
STOP

In [ ]:
#在庫切れのみ


#モジュールのインポート
import os
import shutil
import glob
import pandas as pd
import numpy as np

#設定
delivery_line = 921 #送料ライン
low_price = 1498 #最低価格
delivery = 100 #国内配送料
profit_rate_nomal = 1.15 #利益率
profit_rate_sale = 1.2 #利益率
tax = 1.04 #関税・消費税
commission = 1-0.08 #手数料

#ファイルの読み込み
file_path = glob.glob('./在庫csv/*txt.csv')[-1]
df = pd.read_csv(file_path, encoding="shift_jis")

df = df[df['最安値[FBA]'] == 0]

#価格の計算
fba = df['最安値[FBA]']
df.loc[(fba != 0) & (fba > delivery_line) , 'price'] = (
    (((fba*profit_rate_nomal*tax+delivery)/commission)//100*100))

df.loc[(df['price'] > low_price) & (df['price'] < 9998), 'price'] = (
    df['price']+(1-df['price']//100%2)*100+98)

df.loc[df['price'] > 9998, 'price'] = (
    df['price']//1000*1000+(1-df['price']//1000%2)*1000+980)

df.loc[df['price'] < 1598, 'price'] = 1598
df.loc[(fba <= 921) & (fba != 0), 'price'] = low_price
df.loc[fba == 0, 'price'] = 0

df['sale-price'] = ''
df['sale-period-start'] = ''
df['sale-period-end'] = ''


#小文字kk00化
ser_code = df['ASIN'].str.lower() + ('kk00')
ser_price = df['price'].astype('int')
ser_sale_price = df['sale-price']
ser_sale_period_start = df['sale-period-start']
ser_sale_period_end = df['sale-period-end']



#series作成
df_00 = pd.DataFrame(data={
    'code':ser_code,
    'price':ser_price,
    'sale-price':ser_sale_price,
    'sale-period-start':ser_sale_period_start,
    'sale-period-end':ser_sale_period_end
})

df_00.loc[df_00['price'] == 0, 'display'] = 0
df_00.loc[df_00['price'] != 0, 'display'] = 1
ser_display = df_00['display'].astype('int')
df_00.loc[df_00['price'] == 0, 'lead-time-instock'] = 3
df_00.loc[df_00['price'] != 0, 'lead-time-instock'] = 1
ser_lead_time_instock = df_00['lead-time-instock'].astype('int')
#送料あり[3]、送料なし[1]
df_00.loc[df_00['price'] <= 1499, 'delivery'] = 3
df_00.loc[df_00['price'] > 1499, 'delivery'] = 1
ser_delivery = df_00['delivery'].astype('int')


#DataFrame作成
df_01 = pd.DataFrame(data={
    'code':ser_code,
    'price':ser_price,
    'sale-price':ser_sale_price,
    'sale-period-start':ser_sale_period_start,
    'sale-period-end':ser_sale_period_end,
    'display':ser_display,
    'lead-time-instock':ser_lead_time_instock,
    'delivery':ser_delivery
})


#0の空白化
df_01.loc[df_00['price'] == 0, 'price'] = ''


#ファイス分割
for i in range(len(df_01)//100000+1):
    df_01[(i+1)*100000-100000:(i+1)*100000].to_csv(
        path_or_buf=f'./分割ファイル/変更_{i+1}.csv', 
        index=False, encoding="shift_jis")
    

#在庫切れの割合表示
df_bool = (df['最安値[FBA]'] == 0)
zaikogire_ritu = df_bool.sum()/len(df_bool)*100
print(f'{zaikogire_ritu:.1f}')
print(file_path)


